In [1]:
import os
import opendatasets as od
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from etl import ELTPipline

In [2]:
dataset = 'https://www.kaggle.com/competitions/competitive-data-science-predict-future-sales/data'
od.download(dataset)

Skipping, found downloaded files in ".\competitive-data-science-predict-future-sales" (use force=True to force download)


In [3]:
file_paths = os.listdir('./competitive-data-science-predict-future-sales')

In [4]:
pipeline = ELTPipline()
df_s = list(map(lambda path: pipeline.exctraction(path), file_paths))
for df in df_s:
    pipeline.transform(df)

'''
df_items = pipeline.exctraction(file_paths[0])
df_item_categories = pipeline.exctraction(file_paths[1])
df_sales_train = pipeline.exctraction(file_paths[2])
df_sample_submission = pipeline.exctraction(file_paths[3])
df_shops = pipeline.exctraction(file_paths[4])
df_test = pipeline.exctraction(file_paths[5])
'''

'\ndf_items = pipeline.exctraction(file_paths[0])\ndf_item_categories = pipeline.exctraction(file_paths[1])\ndf_sales_train = pipeline.exctraction(file_paths[2])\ndf_sample_submission = pipeline.exctraction(file_paths[3])\ndf_shops = pipeline.exctraction(file_paths[4])\ndf_test = pipeline.exctraction(file_paths[5])\n'

In [5]:
df_s[2]['date'] = pipeline.data_format(df_s[2]['date'])

In [7]:
df_s[2] = pipeline.outliers(df_s[2], group='item_id', val = 'item_price')


In [ ]:
df_shops.head()

NameError: name 'df_shops' is not defined

In [ ]:
df_sales_train['date'] = pd.to_datetime(df_sales_train['date'], format = '%d.%m.%Y')

In [ ]:
df_sales_train = pd.read_csv('')
df_sales_train['date'].info()

NameError: name 'df_sales_train' is not defined